<a href="https://colab.research.google.com/github/Nouha50215/NLP-Assignment-1/blob/main/Nlp_Assgt1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dataset

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("lakshmi25npathi/imdb-dataset-of-50k-movie-reviews")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'imdb-dataset-of-50k-movie-reviews' dataset.
Path to dataset files: /kaggle/input/imdb-dataset-of-50k-movie-reviews


Load the data

In [ ]:
import pandas as pd

data = pd.read_csv(f"{path}/IMDB Dataset.csv")
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


Convert labels to binary

In [ ]:
data["label"] = data["sentiment"].map({"positive": 1, "negative": 0})

Install & import sentiment tools

In [ ]:
pip install nltk textblob vaderSentiment

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 8.4 MB/s eta 0:00:00


In [ ]:
import nltk
nltk.download("vader_lexicon")

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textblob import TextBlob

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [ ]:
vader = SentimentIntensityAnalyzer()

TextBlob feature

In [ ]:
def extract_features(text):
    vader_scores = vader.polarity_scores(text)
    blob_polarity = TextBlob(text).sentiment.polarity

    return [
        vader_scores["neg"],
        vader_scores["neu"],
        vader_scores["pos"],
        vader_scores["compound"],
        blob_polarity
    ]


In [ ]:
import numpy as np

X = np.array([extract_features(text) for text in data["review"]])
y = data["label"].values


Train / test split

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


Feature scaling (important for MLP)

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


Build the Multi-Layer Perceptron classifier

In [ ]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(
    hidden_layer_sizes=(64, 32),
    activation="relu",
    solver="adam",
    max_iter=20,
    random_state=42
)


Train

In [ ]:
mlp.fit(X_train, y_train)

/usr/local/lib/python3.12/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(hidden_layer_sizes=(64, 32), max_iter=20, random_state=42)

Evaluation

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

y_pred = mlp.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.776

Classification Report:
               precision    recall  f1-score   support

           0       0.77      0.80      0.78      5000
           1       0.79      0.76      0.77      5000

    accuracy                           0.78     10000
   macro avg       0.78      0.78      0.78     10000
weighted avg       0.78      0.78      0.78     10000



Save the model and scaler

In [ ]:
import joblib

joblib.dump(mlp, "mlp_sentiment_model.pkl")
joblib.dump(scaler, "scaler.pkl")

['scaler.pkl']